In [1]:
import re
import json

In [2]:
def readFile(fn):
    fp = file(fn)
    content = fp.read()
    fp.close()
    return content

content = readFile('qa_test.txt')

In [4]:
def find_right_next(s, i, n, char):
    if i == len(s):
        return i

    if s[i] == '{':
        return find_right_next(s, i + 1, n + 1, char)
    elif s[i] in '%' + char and n == 0:
        return i
    elif s[i] == '}' and n > 0:
        return find_right_next(s, i + 1, n - 1, char)
    else:
        return find_right_next(s, i + 1, n, char)

In [6]:
start = content.find("{%")
end = find_right_next(content, start, 0, '\n')
s = content[start:end]
print end
print s

543
{%process
|下面我们开始
@a
@b:`x+y`
@a,b:c:$w^Tx+b=0$
@b:d:![x](http://blog.pluskid.org/wp-content/uploads/2010/09/Hyper-Plane.png)
@d:e:`x/y`
#请把w作为系数写在x前面
#但维度是否对上，没对上要如何对上？#偏置项都是kx+b哪见过kx-b了#答案w^Tx + b = 0
%}


In [16]:
def parse_answer(line, p, quiz_type):
    """
    1. 需要在js里判断':'和'@' 中间不存在 ','，一个答案只写一次，独占一空
    1. 如果存在两个 : 需要在js里判断中间存在 ',' 而不是 '，'
    """
    obj = re.findall(p, line, re.M)
    if not obj:
        return None

    result = None
    lists = [e.replace('\n', '').replace('\r', '') for e in obj[0].split(p[0])]
    options = []
    for l in lists:
        if quiz_type != 'process':
            break

        t = [e.replace('\n', '').replace('\r', '') for e in l.split(':')]
        lt = len(t)
        print l, t, lt,
        answer_map = {}
        if lt == 1:
            answer_map  = {t[0]: ([], '')}
        if lt == 2:
            pre = t[0].split(',')
            if t[1][0] == '$' or t[1][0] == '!' or t[1][0] == '`':
                answer_map = {t[0]: ([], t[1])}
                options.append(t[1])
            else:
                answer_map = {t[0]: (t[1].split(','), '')}
        if lt == 3:
            answer_map  = {t[0]: (t[1].split(','), t[2])}
            options.append(t[2])

        print answer_map
        if not result:
            result = answer_map
            result['options'] = options
            continue

        for e in answer_map:
            result[e] = answer_map[e]

    if not result:
        result = lists

    return result


In [17]:
quiz_type = re.search('^{%(\w+)|', s, re.M).group(1)

answer = parse_answer(s, '@([^#]*)', quiz_type)
print quiz_type
print answer

a ['a'] 1 {'a': ([], '')}
b:`x+y` ['b', '`x+y`'] 2 {'b': ([], '`x+y`')}
a,b:c:$w^Tx+b=0$ ['a,b', 'c', '$w^Tx+b=0$'] 3 {'a,b': (['c'], '$w^Tx+b=0$')}
b:d:![x](http://blog.pluskid.org/wp-content/uploads/2010/09/Hyper-Plane.png) ['b', 'd', '![x](http', '//blog.pluskid.org/wp-content/uploads/2010/09/Hyper-Plane.png)'] 4 {}
d:e:`x/y` ['d', 'e', '`x/y`'] 3 {'d': (['e'], '`x/y`')}
process
{'a': ([], ''), 'b': ([], '`x+y`'), 'a,b': (['c'], '$w^Tx+b=0$'), 'options': ['`x+y`', '$w^Tx+b=0$', '`x/y`'], 'd': (['e'], '`x/y`')}


In [ ]:
start = content.find("{%")
lists = []
while start < len(content):
    end = find_right_next(content, start, 0, '\n')
    s = content[start:end]
    lists.append(s)
    quiz_type = re.search('^{%(\w+)|', s, re.M).group(1)
    answer, comment = parse_quiz(s, quiz_type)
    answers.append(answer)
    comments.append(comment)
    start = content.find("{%", end)
    if start <= 0:
        break